In [50]:
import glob
import numpy as np
import csv
from sklearn.cluster import KMeans

fileNames = glob.glob('2019Proj2_train/*.txt')
# print(fileNames)


def fileReader(file,num_clusters = 50):
    '''
    This will read the file, and output a discretized value, based on Kmeans clustering
    
    input:
    file: Filename of training data used
    num_clusters: number of discrete values based on Kmeans clustering
    
    output:
    kmean.labels: [0,num_clusters) - exclusive of num_clusters number!
    '''
    a = []
    # Creating a list, appending to list, and then making a numpy array
    
    
    with open(file, 'r') as f:
        reader = csv.reader(f, delimiter='\t')
        for row in reader:
            a.append(row)
    tmp = np.array(a)
    kmeans = KMeans(n_clusters=num_clusters, random_state=0).fit(tmp)
    return kmeans.labels_



# This should be trained on all of the data. After doing this I need to make the silos of data

num_clusters = 50 # number of discrete values

beat_three = fileNames[0:5]; beat3_data = np.array([fileReader(beat3) for beat3 in beat_three])
beat_four = fileNames[5:10]; beat4_data = np.array([fileReader(beat4) for beat4 in beat_four])
circle = fileNames[10:15]; circle_data = np.array([fileReader(circ) for circ in circle])
eight = fileNames[15:20]; eight_data = np.array([fileReader(e) for e in eight])
inf = fileNames[20:25]; inf_data = np.array([fileReader(i) for i in inf])
wave = fileNames[25:30]; wave_data = np.array([fileReader(wv) for wv in wave])

# print(wave_data[:10], "\nlast couple: \n", wave_data[-10:])

print(inf_data[0][:10])

[35 35 35 35 35 35 35 35 35 35]


In [56]:
print(np.max(inf_data[0]))


# Have one array with all the data, and the indexes will tell me where to start/stop


# This function was only needed for an older version
# def findIndex(*args):
#     # input: List of filenames
#     i = []
#     ind = 0
#     for arg in args:
#         i.append(ind)
#         ind += len(fileReader(arg))
        
#     return i


def dictCreator(names,indexes):
    d = {}
    rd = {}
    for name,index in zip(names,indexes):
        d[index] = name
        rd[name] = index
    return d,rd

num_clusters = 50
Hidden_states = {'beat_three','beat_four','circle','eight','inf','wave'}
observation_states = set(range(num_clusters))

# print(val.shape)
# print(val[1:10000])



49


In [37]:
# Finally starting HMM
# print(indDict)

from scipy.special import logsumexp
import math






class Model():
        
        def __init__(self, num_HiddenState, num_observationClass):
            # Define the initial (A,B, and pi)
            self.num_HiddenState = num_HiddenState
            self.num_observations = num_observationClass
            self.P = self.prior(num_HiddenState)
            self.E = self.Emission(num_HiddenState, num_observationClass)
            self.T = self.Transition(num_HiddenState)
            self.hidden = num_HiddenState
            self.observed = num_observationClass
            print("Prior",self.P.shape)
            print("Emission",self.E.shape)
            print("Transition",self.T.shape)
    
    
        def prior(self, num_HiddenState):
            p = np.divide(np.ones(num_HiddenState),num_HiddenState)

#             p[0] = 1
            return p

        def Emission(self, num_HiddenState, num_observationClass):
            e = np.zeros((num_HiddenState,num_observationClass))
            e[:,:] = 1/num_observationClass


            return e

        def Transition(self, num_HiddenState):
            '''
            First, we will try creating an ergodic transition matrix. Next, will try a left-right model
            '''
            t = np.full((num_HiddenState,num_HiddenState),1/num_HiddenState)
            
            


            # This will create a left-right matrix. Will try this later, but for now ergodic
            
#             for i in range(num_HiddenState-2):
#                 t[i,i] = 1/3
#                 t[i,i+1] = 1/3
#                 t[i,i+2] = 1/3

#             # Final initializations
#             t[num_HiddenState-2,num_HiddenState-2] = 1/2
#             t[num_HiddenState-2,num_HiddenState-1] = 1/2

#             t[num_HiddenState-1,num_HiddenState-1] = 1


            return t
        
        def forwardInduction(self, sequence, reverse = False):
            print("too much T",self.T.shape)
            print("Emissions", self.E.shape)
            alpha = np.zeros((len(self.P),len(sequence)))
            alpha[:,0] = self.P * self.E[:,0]
#             print(alpha)

            # Now the real fun begins
            for i,state in enumerate(sequence,1):
                alpha[:,i] = logsumexp(np.multiply(alpha[:,i-1] , self.E[:,i]))
            print(alpha)
           
            
            
#             # first pass
#             alpha[0] = self.P*self.Emission
            
    
            
            
#             for state in arange(num_states):
#                 t_update = logsumexp(t[state] * t[state][]) * log() # Recommended by Anthony for more efficiency
        

            
            








model = Model(4,5)


def initialization(Model, sequence):
    '''
    The forward calculation of the algorithm
    '''
    print(sequence.shape)
    alpha = np.zeros((Model.num_HiddenState,len(sequence)))
    alpha[:,0] = np.multiply(Model.P, Model.E[:,0])
    print(alpha.shape)
    
    
    for t,time in enumerate(sequence[1:],1):
        
        for j,state_out in enumerate(range(Model.num_HiddenState)): 
            alpha[j,t] = logsumexp([np.multiply(alpha[i,t-1],Model.T[j,i]) for i in range(Model.num_HiddenState)]) \
            + math.log(0.2) # I need to make this actually relate to the observations.            
    print("alpha: ",alpha)
    return alpha

def initializeBeta(Model,sequence):
    '''
    The backwards calculation of the forward-backward algorithm
    '''
    beta = np.zeros((Model.num_HiddenState,len(sequence)))
    beta[:,-1] = np.ones((Model.num_HiddenState,))
    
    for t,_ in enumerate(sequence[:-1],1):
        t = len(sequence) - 1 - t
        
        for j,_ in enumerate(range(Model.num_HiddenState)): 
            beta[j,t] = logsumexp([np.multiply(beta[i,t+1],Model.T[j,i]) for i in range(Model.num_HiddenState)]) \
            + math.log(0.2) # I need to make this actually relate to the observations.
            
    print("beta: ",beta)
    return beta

def optimalseq(Model, sequence, alpha, beta):
    '''
    This will return the optimal state sequence given these observations.
    
    sequence: refers to observation sequence
    
    returns:
    optSeq: This is the optimal sequence
    
    '''
    gamma = np.zeros((Model.num_HiddenState,len(sequence)))
    
    for t, _ in enumerate(sequence):
        normalization = np.sum(np.multiply(alpha[:,t],beta[:,t]))
        tmp = np.multiply(alpha[:,t],beta[:,t])
        gamma[:,t] = np.divide(tmp,normalization)
        
    optSeq = np.argmax(gamma,axis=0)
    print("optSeq: ", optSeq)
    
    return optSeq, gamma

def EM(Model, sequence, alpha, beta, optimal_sequence,gamma):
    '''
    The output of xi should be the size of the sequence
    '''
#     xi = np.zeros((len(optimal_sequence)-1,))
    xi = np.zeros((Model.num_HiddenState, Model.num_HiddenState,len(optimal_sequence)-1))
#     print(xi.shape)
    
    for t in range(len(optimal_sequence)-1):
        # Hidden states that are inferred for current and one ahead in time
        ot = optimal_sequence[t]
        ot1 = optimal_sequence[t+1]
        
        # Observed sequences
        st = sequence[t]
        st1 = sequence[t+1]


        # Very long list comprehension, that forms that array
        tmp = np.array([alpha[i,t] * Model.T[i,j] * Model.E[optimal_sequence[t+1],j] * beta[j, optimal_sequence[t+1]] \
        for j in range(Model.num_HiddenState) for i in range(Model.num_HiddenState)])
        normalization = np.sum(tmp)

        xi[:,:,t] = np.reshape(tmp/normalization,(4,4))
        
        
#         This gamma isn't used. Hope that is ok!
    newgamma = np.sum(xi,axis= 0)
    
    
    
    
    # Update initial probability
    print("updated P: ",gamma[:,0])
    Model.P = gamma[:,0]
    
    
    
    # Update transition probabilities
    
    Model.T = np.sum(xi,axis = 2) / np.sum(gamma[:,:-1], axis = 1)
#         print("Updated T: ",np.sum(xi,axis = 2)[i,:] / np.sum(gamma[:,:-1], axis = 0)[i])
    print("tmp: ",Model.T)

    
    
    
    # Update emission probabilities
    print("Updated E: ", np.sum(gamma, axis = 0) / np.sum(gamma, axis = 0))
#     Model.E = np.sum(gamma, axis = 0) / np.sum(gamma, axis = 0)

    return xi,gamma
       



def baumWelch(Model,sequence):
    '''
    Describes doing the forward-backward calculation, followed by determining the optimal sequence that is 
    described by this
    
    Model: the HMM model that is used, which includes in hidden_states, # different observations
    sequence: The sequence that acts as your input
    '''
    o = set(sequence)
    alpha = initialization(Model,sequence)
    beta = initializeBeta(Model,sequence)
    optSeq, gamma = optimalseq(Model, sequence, alpha, beta)
    xi,_ = EM(Model, sequence, alpha,beta,optSeq, gamma)
        
    
baumWelch(model,np.array([5,6,7,8,9,8]))



Prior (4,)
Emission (4, 5)
Transition (4, 4)
(6,)
(4, 6)
alpha:  [[ 0.05       -0.21061758 -0.27532622 -0.29109516 -0.29483303 -0.29567956]
 [ 0.05       -0.21061758 -0.275309   -0.29080338 -0.2942946  -0.29504598]
 [ 0.05       -0.21056543 -0.27438126 -0.28908058 -0.29226969 -0.29293723]
 [ 0.05       -0.21040723 -0.271742   -0.28447367 -0.28703957 -0.28755354]]
beta:  [[-0.29460893 -0.29037678 -0.27394143 -0.21218966  0.03668621  1.        ]
 [-0.29404905 -0.28980381 -0.27194782 -0.20395087  0.03668621  1.        ]
 [-0.2920351  -0.28804125 -0.27030188 -0.19875994  0.05778625  1.        ]
 [-0.28684262 -0.28349186 -0.26690442 -0.18785933  0.13423047  1.        ]]
optSeq:  [0 0 0 0 3 0]
updated P:  [0.25233398 0.25185444 0.25012948 0.2456821 ]
tmp:  [[0.36951788 0.         0.         0.        ]
 [0.3659314  0.37006687 0.         0.        ]
 [0.36202855 0.3661205  0.51794297 0.        ]
 [0.         0.35686965 0.5048662  0.83054755]]
Updated E:  [1. 1. 1. 1. 1. 1.]
